## Requirement Install


In [ ]:
!pip install pandas numpy scikit-learn
!pip install konlpy
!pip install torch             # 또는 GPU:
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu117
!pip install sentence-transformers
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [ ]:
!pip install sentence-transformers

## 음식점 리뷰 키워드 추출

### Output File Name :
Sample_음식점/음식점_all_reviews_with_keywords_sentiment_all.csv

In [ ]:
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from glob import glob
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample_음식점/음식점_리뷰크롤링_ALL.csv')

df = pd.concat([
    pd.read_csv(fp, encoding='utf-8', engine='python')
    for fp in file_paths
], ignore_index=True)

# 2) 리뷰 컬럼이름 통일 & 빈 값 제거
if '리뷰내용' not in df.columns:
    raise ValueError("컬럼 '리뷰내용'이 없습니다.")
df = df[['리뷰내용']].dropna(subset=['리뷰내용'])
df = df.rename(columns={'리뷰내용':'review'})
df['review'] = df['review'].astype(str)
# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기
okt = Okt()
def preprocess(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)  # 특수문자 제거
    text = re.sub(r'\d+', ' ', text)      # 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    pos = okt.pos(text, stem=True)
    return [
        word for word, tag in pos
        if tag in ['Noun','Adjective','Verb'] and word not in STOPWORDS
    ]

# 4) 키워드 추출 함수 (SBERT)
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask')
def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    # 전처리 & 토크나이즈
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []

    # TF-IDF 로 후보 만들기 (에러 방어)
    try:
        tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
        candidates = tfidf.get_feature_names_out()
    except ValueError:
        return []

    if len(candidates) == 0:
        return []

    # 실제 뽑을 키워드 개수
    k = min(top_k, len(candidates))

    # 문장과 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 상위 k개
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    topk_idx = scores.topk(k).indices.cpu().numpy()
    return [candidates[i] for i in topk_idx]

# 5) 감성분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device=-1
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['review','keywords','sentiment']].head())

# 8) CSV로 저장
df.to_csv('/content/drive/MyDrive/Sample_음식점/음식점_all_reviews_with_keywords_sentiment_all.csv', index=False)


## 숙박 리뷰 키워드 추출

Output File Name :
Sample/Store Name + 숙박_all_reviews_with_keywords_sentiment.csv


In [ ]:
# -*- coding: utf-8 -*-
# -*- coding: utf-8 -*-
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # BLAS 스레드 제한

import re
import pandas as pd
import numpy as np
from glob import glob
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample/네이버 지도 방문자 리뷰 크롤러_숙박_*.csv')

df = pd.concat([
    pd.read_csv(fp,
                usecols=['리뷰내용','가게이름','카테고리','전체평점','방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간'],
                encoding='utf-8',    # 필요에 따라 'cp949' 등으로 변경
                engine='python')
    for fp in file_paths
], ignore_index=True)

# 2) 컬럼명 통일 및 결측치 제거
#   - 리뷰가 없는 행은 제거하지만, 가게명은 그대로 보존합니다.
df = df.dropna(subset=['리뷰내용'])
df = df.rename(columns={
    '리뷰내용': 'review'
})
df['review'] = df['review'].astype(str)

# 2) 불용어 사전
STOPWORDS = set(['이','가','은','는','도','에','와','과','로','으로','의','를','을','하다'])

# 3) 형태소 분석기
okt = Okt()
def preprocess(text: str) -> str:
    text = re.sub(r'[^\w\s]', ' ', text)  # 특수문자 제거
    text = re.sub(r'\d+', ' ', text)      # 숫자 제거
    return re.sub(r'\s+', ' ', text).strip()

def tokenize(text: str) -> list[str]:
    pos = okt.pos(text, stem=True)
    return [
        word for word, tag in pos
        if tag in ['Noun','Adjective','Verb'] and word not in STOPWORDS
    ]

# 4) 키워드 추출 함수 (SBERT)
KW_MODEL = SentenceTransformer('jhgan/ko-sroberta-multitask')
def extract_keywords(doc: str, top_k: int = 5) -> list[str]:
    # 전처리 & 토크나이즈
    clean = preprocess(doc)
    toks  = tokenize(clean)
    if not toks:
        return []

    # TF-IDF 로 후보 만들기 (에러 방어)
    try:
        tfidf = TfidfVectorizer(ngram_range=(1,1)).fit([' '.join(toks)])
        candidates = tfidf.get_feature_names_out()
    except ValueError:
        return []

    if len(candidates) == 0:
        return []

    # 실제 뽑을 키워드 개수
    k = min(top_k, len(candidates))

    # 문장과 후보 임베딩
    emb_doc = KW_MODEL.encode(doc, convert_to_tensor=True)
    emb_kw  = KW_MODEL.encode(candidates, convert_to_tensor=True)

    # 코사인 유사도 상위 k개
    scores = util.pytorch_cos_sim(emb_doc, emb_kw)[0]
    topk_idx = scores.topk(k).indices.cpu().numpy()
    return [candidates[i] for i in topk_idx]

# 5) 감성분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='monologg/koelectra-base-v3-discriminator',
    tokenizer='monologg/koelectra-base-v3-discriminator',
    device=-1
)

# 6) DataFrame에 적용
df['cleaned']   = df['review'].map(preprocess)
df['tokens']    = df['cleaned'].map(tokenize)
df['keywords']  = df['review'].map(lambda x: extract_keywords(x, top_k=5))
df['sentiment'] = df['review'].map(lambda x: sentiment(x)[0]['label'])

# 7) 결과 확인
print(df[['가게이름','카테고리','전체평점','방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간','review','keywords','sentiment']].head())

# 8) CSV로 저장
df.to_csv('/content/drive/MyDrive/Sample/Store Name + 숙박_all_reviews_with_keywords_sentiment.csv', index=False)


## 상위 5개 키워드 추출

In [ ]:
import pandas as pd
from collections import Counter

# 리뷰 데이터 (예시)
# df에는 최소 다음 컬럼들이 있어야 합니다: '가게이름', 'keywords'
# 'keywords'는 각 리뷰에 대해 추출된 키워드 리스트 형태 (예: ['조용하다', '깨끗하다', '가깝다'])

# 1. 결측치 제거 및 확인
df_keywords = data.dropna(subset=['가게이름', 'keywords'])
df_keywords['keywords'] = df_keywords['keywords'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# 2. 장소별 키워드 누적
store_keywords = df_keywords.groupby('가게이름')['keywords'].sum()

# 3. 각 장소에서 상위 5개 키워드 추출
top_keywords_per_store = store_keywords.apply(lambda x: [kw for kw, _ in Counter(x).most_common(8)])

# 4. DataFrame으로 변환
top_keywords_df = top_keywords_per_store.reset_index()
top_keywords_df.columns = ['가게이름', '상위_키워드']
top_keywords_df.to_csv('/content/drive/MyDrive/Sample/25_05_10_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv', index=False)

# 결과 확인
print(top_keywords_df.head(50))


##관광지 키워드 이용 추천 시스템



In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util
from math import radians, sin, cos, sqrt, atan2
import lightgbm as lgb
import torch

# 1) 데이터 불러오기 -------------------------------------------------------

meta_df = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv',
    usecols=[
      'contents_id','contents_label','title','address','road_address','tag',
      'introduction','latitude','longitude',
      '평일오픈시간','평일클로즈시간','주말오픈시간','주말클로즈시간'
    ]
)
reviews_df = pd.read_csv('/content/drive/MyDrive/Sample_관광지/Store Name + 관광지_all_reviews_with_quarter.csv',
    usecols=[
      '가게이름','카테고리','전체평점','방문자리뷰',
      '리뷰작성자','review','이런_점이_좋아요','방문시간',
      'cleaned','tokens','keywords','sentiment','quarter'
    ]
)

meta_map = meta_df[['contents_id','title']].rename(columns={'title':'가게이름'})
reviews_df = reviews_df.merge(meta_map, on='가게이름', how='left')

# 2) 장소별 키워드 집계 -------------------------------------------------------
def aggregate_top_keywords(df, place_col='contents_id', kw_col='keywords', top_n=5):
    # 리뷰별 keywords 리스트 explode
    exploded = df[[place_col, kw_col]].explode(kw_col).dropna(subset=[kw_col])
    counts = (
      exploded
      .groupby([place_col, kw_col])
      .size()
      .reset_index(name='cnt')
    )
    # 장소별 상위 top_n 키워드만 추출
    topk = (
      counts
      .sort_values([place_col,'cnt'], ascending=[True,False])
      .groupby(place_col)
      .head(top_n)
      .groupby(place_col)[kw_col]
      .apply(list)
      .reset_index()
      .rename(columns={kw_col:'top_keywords'})
    )
    # 리스트를 공백으로 합쳐서 한 문장으로
    topk['keyword_text'] = topk['top_keywords'].map(lambda kws: " ".join(kws))
    return topk[[place_col,'keyword_text']]

store_kw = aggregate_top_keywords(reviews_df, place_col='contents_id', kw_col='keywords', top_n=5)
meta_df = meta_df.merge(store_kw, on='contents_id', how='left').fillna({'keyword_text':''})


# 3) SBERT 임베딩 & 유사도 ----------------------------------------------------
sbert = SentenceTransformer('jhgan/ko-sroberta-multitask')
# 장소별 키워드 문장 임베딩 (tensor list)
meta_df['kw_emb'] = list(
    sbert.encode(
      meta_df['keyword_text'].tolist(),
      convert_to_tensor=True
    )
)
# 1) 모든 임베딩을 (N, D) 형태로 쌓기
embs = torch.stack(meta_df['kw_emb'].tolist(), dim=0)

# 2) N×N 코사인 유사도 행렬 계산
sim_matrix = util.pytorch_cos_sim(embs, embs).cpu().numpy()


# 4) Haversine 거리 계산 -----------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구 반경 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

coords = meta_df[['latitude','longitude']].to_numpy()
n = len(coords)
dist_matrix = np.zeros((n,n), dtype=float)
for i in range(n):
    for j in range(i+1, n):
        d = haversine(*coords[i], *coords[j])
        dist_matrix[i,j] = dist_matrix[j,i] = d


# 5) Co-Review Count 생성 ----------------------------------------------------
# 같은 (year,quarter,user_id)에 리뷰를 남긴 장소 쌍 수를 센다.
pair_counter = Counter()
grp = (
    reviews_df
    .dropna(subset=['contents_id','review'])
    .groupby(['quarter','리뷰작성자'])
    ['contents_id']
    .apply(lambda lst: sorted(set(lst)))
)
for places in tqdm(grp, desc="building co-review"):
    for i, j in combinations(places, 2):
        pair_counter[(i,j)] += 1

co_pairs = [
    {'place1': i, 'place2': j, 'co_count': cnt}
    for (i,j), cnt in pair_counter.items()
]
co_df = pd.DataFrame(co_pairs)


# 6) 피처 테이블 생성 --------------------------------------------------------
# place_id → meta_df 인덱스 매핑
place2idx = {pid: idx for idx, pid in enumerate(meta_df['contents_id'])}

def make_feature_table(co_df, meta_df):
    rows = []
    for _, r in tqdm(co_df.iterrows(), total=len(co_df), desc="feature table"):
        i, j = r['place1'], r['place2']
        if i not in place2idx or j not in place2idx:
            continue
        idx_i, idx_j = place2idx[i], place2idx[j]
        rows.append({
            'place1': i,
            'place2': j,
            'co_count': r['co_count'],
            'dist_km':    dist_matrix[idx_i, idx_j],
            'sbert_sim':  sim_matrix[idx_i, idx_j],
        })
    return pd.DataFrame(rows)

feat_df = make_feature_table(co_df, meta_df)


# 7) LightGBM 학습 파이프라인 예시 ------------------------------------------
# 회귀(regression) 혹은 분류(classification) 태스크로 바꾸셔도 됩니다.
X = feat_df.drop(columns=['place1','place2','co_count'])
y = feat_df['co_count']

lgb_train = lgb.Dataset(X, y)
params = {
    'objective':'regression',
    'metric':'rmse',
    'learning_rate':0.1,
}
gbm = lgb.train(params, lgb_train, num_boost_round=200)


# 8) 결과 저장 ---------------------------------------------------------------
# 메타: 키워드 문장만 남기고 embedding은 삭제
meta_df.drop(columns=['kw_emb'], inplace=True)
meta_df.to_csv('meta_with_kw.csv', index=False)
feat_df.to_csv('item_cf_features.csv', index=False)

print("파이프라인 완료")

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
meta_df = pd.read_csv('/content/drive/MyDrive/Meta Data/jeju_tour_spot.csv',
    usecols=[
        'contents_id','title',
        'tag','introduction'
    ]
)

#    키워드 텍스트 불러오기
kw_df = pd.read_csv('/content/drive/MyDrive/Sample/25_05_10_Store Name + 숙박_all_reviews_with_keywords_sentiment.csv',
    usecols=['가게이름','상위_키워드']
)

kw_df = kw_df.rename(columns={
    '가게이름':     'title',
    '상위_키워드': 'keyword_text'
})
# 병합 (LEFT JOIN)
df = meta_df.merge(kw_df, on='title', how='left')

# 누락된 키워드는 빈 문자열로
df['keyword_text'] = df['keyword_text'].fillna('')

# tag와 keyword_text 합쳐서 추천용 텍스트 생성
df['combined_text'] = (
    df['tag'].fillna('') + ' ' + df['keyword_text']
).str.strip()


# 1) SBERT 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 2) 사용자가 입력한 키워드 리스트 예시
user_kw = ['산책', '해안도로','드라이브']
user_sent = " ".join(user_kw)
emb_user = model.encode(user_sent, convert_to_tensor=True)


# 3) 장소별 combined_text 임베딩 (한 번만 수행)
emb_corpus = model.encode(
    df['combined_text'].tolist(),
    convert_to_tensor=True
)

cos_scores = util.pytorch_cos_sim(emb_user, emb_corpus)[0]  # tensor of length N
top_k = 10
top_results = torch.topk(cos_scores, k=top_k)

# 6) 결과 출력
top_indices = top_results.indices.cpu().numpy()
top_scores  = top_results.values.cpu().numpy()
recommendations = df.iloc[top_indices].copy()
recommendations['score'] = top_scores

# 최종 추천 리스트 보기
print(recommendations[['contents_id','title','score']])

## Co-Review Data

### 해야할 일 :
- 관광지 + 음식점 파일로 구현해서 다시 돌리기


Output Filename :
/Sample/관광지_co_review_pairs.csv



In [ ]:
from glob import glob

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample_관광지/네이버 지도 방문자 리뷰 크롤러_관광지_*.xlsx')

df = pd.concat([
    pd.read_excel(
        fp,
        usecols=[
            '가게이름','카테고리','전체평점',
            '방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간'
        ],
        engine='openpyxl'     # openpyxl 엔진 사용
    )
    for fp in file_paths
], ignore_index=True)

# 방문시간을 문자열로 변환
df['방문시간'] = df['방문시간'].astype(str)

# 연도 포함 여부 판단: '^\d{2}\.\d{1,2}\.\d{1,2}' 형태인 경우 → 연도 있음
has_year = df['방문시간'].str.contains(r'^\d{2}\.\d{1,2}\.\d{1,2}')

# 연도 없는 데이터에는 기본 연도 '24' 추가 (또는 원하는 연도)
df.loc[~has_year, '방문시간'] = '24.' + df.loc[~has_year, '방문시간']

# 날짜 문자열만 추출
df['date_str'] = df['방문시간'].str.extract(r'(\d{2}\.\d{1,2}\.\d{1,2})')[0]

# datetime 변환
df['방문시간'] = pd.to_datetime(
    df['date_str'],
    format='%y.%m.%d',
    errors='coerce'
)

# 분기만 추출
df['quarter'] = df['방문시간'].dt.quarter

# 불필요한 중간 컬럼 제거
df.drop(columns=['date_str'], inplace=True)

#print(df[['가게이름','방문시간','quarter']].head(20))

from itertools import combinations
from collections import Counter

# 1. 필요 컬럼만 추출
df = df[['가게이름', '리뷰작성자', 'quarter']].dropna()

# 2. 사용자-분기 기준 그룹핑하여 장소 목록 생성
user_quarter_group = df.groupby(['리뷰작성자', 'quarter'])['가게이름'].apply(list)

# 3. 각 그룹에서 장소 쌍 조합 생성
co_review_pairs = []

for places in user_quarter_group:
    # 장소가 2개 이상인 경우만 조합
    if len(set(places)) >= 2:
        pairs = combinations(sorted(set(places)), 2)
        co_review_pairs.extend(pairs)

# 4. 장소 쌍별 등장 횟수 계산
pair_counts = Counter(co_review_pairs)

# 5. 결과를 DataFrame으로 변환
pair_df = pd.DataFrame([
    {'가게1': p1, '가게2': p2, 'co_review_count': count}
    for (p1, p2), count in pair_counts.items()
])

# 6. co_review 수 내림차순 정렬
pair_df = pair_df.sort_values(by='co_review_count', ascending=False)

# 7. 결과 확인
print(pair_df.head(20))

# 8. 저장 (선택사항)
pair_df.to_csv('/content/drive/MyDrive/Sample/관광지_co_review_pairs.csv', index=False)
